Importieren der benötigten Pakete

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from numpy.linalg import svd
from scipy.optimize import least_squares
import seaborn as sns
from scipy.spatial.transform import Rotation as Rot
from matplotlib import rcParams
import locale
import matplotlib as mpl

# Setze deutsches Locale (ggf. 'de_DE.UTF-8' auf deinem System verfügbar machen)
locale.setlocale(locale.LC_ALL, 'de_DE.utf8')

# Erlaube den Formatierern, das Locale zu nutzen
mpl.rcParams['axes.formatter.use_locale'] = True


Festlegen der Zylinderparameter

In [2]:
# Zylinder-Parameter (konstant für alle Datensätze)
cylinder_start = np.array([1006.279388, 2002.087212,  98.11687])
cylinder_end   = np.array([1006.897308, 1962.093132,  97.81199])
cyl_axis = cylinder_end - cylinder_start
cyl_axis /= np.linalg.norm(cyl_axis)
diameter = 12.922
cylinder_radius = diameter/2
cylinder_height = np.linalg.norm(cylinder_end - cylinder_start)

Funktionen definieren

In [3]:
# Transformation zwischen Quaternionen und Rotationsmatrizen
def quaternion_to_rotation_matrix(q):
    q = np.array(q, dtype=np.float64)
    q /= np.linalg.norm(q)
    w,x,y,z = q
    R = np.array([
        [1-2*(y**2+z**2), 2*(x*y - z*w), 2*(x*z + y*w)],
        [2*(x*y + z*w), 1-2*(x**2+z**2), 2*(y*z - x*w)],
        [2*(x*z - y*w), 2*(y*z + x*w), 1-2*(x**2+y**2)]
    ])
    return R

# Ermittlung der Transformation zwischen zwei Punktwolken mittels Kabsch-Algorithmus
def kabsch_algorithm(P, Q):
    p_centroid = P.mean(axis=0)
    q_centroid = Q.mean(axis=0)
    Pc = P - p_centroid
    Qc = Q - q_centroid
    H = Pc.T @ Qc
    U,S,Vt = svd(H)
    R = (Vt.T @ U.T)
    if np.linalg.det(R) < 0:
        Vt[-1,:] *= -1
        R = Vt.T @ U.T
    t = q_centroid - R @ p_centroid
    return R, t

# Rotationsmatrix aus Euler-Winkeln (Z-Y-X) erstellen	
def rotation_matrix_from_euler(alpha, beta, gamma):
    Rz = np.array([
        [np.cos(alpha), -np.sin(alpha), 0],
        [np.sin(alpha),  np.cos(alpha), 0],
        [0,0,1]
    ])
    Ry = np.array([
        [ np.cos(beta), 0, np.sin(beta)],
        [0,1,0],
        [-np.sin(beta),0, np.cos(beta)]
    ])
    Rx = np.array([
        [1,0,0],
        [0,np.cos(gamma), -np.sin(gamma)],
        [0,np.sin(gamma),  np.cos(gamma)]
    ])
    return Rz @ Ry @ Rx

# Residuenfunktion für die Optimierung
def residual(params, P, Q):
    alpha,beta,gamma,tx,ty,tz = params
    R_opt = rotation_matrix_from_euler(alpha,beta,gamma)
    P_tr = (R_opt @ P.T).T + np.array([tx,ty,tz])
    return (P_tr - Q).ravel()


# Berechnung der Schnittpunkte zwischen Zylinder und Strahl
def compute_intersection_1(pos, dir):
    delta_p = pos - cylinder_start
    v_perp = dir - np.dot(dir, cyl_axis)*cyl_axis
    p_perp = delta_p - np.dot(delta_p, cyl_axis)*cyl_axis
    a = np.dot(v_perp, v_perp)
    b = 2*np.dot(p_perp, v_perp)
    c = np.dot(p_perp,p_perp) - cylinder_radius**2
    delta = b**2 - 4*a*c
    if delta < 0:
        return None
    t0 = (-b - np.sqrt(delta)) / (2*a)
    t1 = (-b + np.sqrt(delta)) / (2*a)
    # Wähle die nähere Lösung
    t = t0 if abs(t0) < abs(t1) else t1
    X = pos + t*dir
    proj = np.dot(X - cylinder_start, cyl_axis)
    if 0 <= proj <= cylinder_height:
        return X
    return None

# Layout-Settings wie gehabt
layout_design = {
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Latin Modern Sans"],

    # Lade lmodern und sansmath, aktiviere serifenlose Mathematik
    "text.latex.preamble": r"""
        \usepackage{lmodern}
        \usepackage{sansmath}
        \sansmath
        \renewcommand{\familydefault}{\sfdefault}
        \usepackage{icomma}
    """,
    
    "font.size": 20,
    "axes.labelsize": 20,
    "axes.titlesize": 12,
    "legend.fontsize": 18,
    "xtick.labelsize": 18,
    "ytick.labelsize": 18,
    "figure.figsize": (6,4),
    "figure.dpi": 300,
    "savefig.bbox": "tight",
    "axes.grid": True,
    "grid.color": "gray",
    "grid.linestyle": "--",
    "grid.linewidth": 0.5,
    "axes.edgecolor": "black",
    "axes.linewidth": 0.8,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.major.size": 5,
    "ytick.major.size": 5,
    "xtick.minor.size": 2.5,
    "ytick.minor.size": 2.5,
}
rcParams.update(layout_design)

def process_dataset(prefix):
    """Verarbeitet einen einzelnen Datensatz mit dem gegebenen Dateipräfix."""
    # 1) Einlesen Kamera- und TS-Daten
    cam_file = f"../data/Transformation_Trockenversuche_18_03_Kamera.csv"
    ts_file  = f"../data/Transformation_Trockenversuche_18_03_TS.csv"

    df_cam = pd.read_csv(cam_file,  sep=';', index_col=0).sort_index()
    df_ts  = pd.read_csv(ts_file,   sep=';', index_col=0).sort_index()

    P = df_cam[['X','Y','Z']].values / 1000
    Q = df_ts [['X','Y','Z']].values

    # 2) Kabsch + Feintuning
    R0, t0 = kabsch_algorithm(P,Q)
    x0 = np.array([0,0,0, *t0])
    res = least_squares(residual, x0, args=(P,Q), loss='huber', f_scale=1.0)
    alpha,beta,gamma,tx,ty,tz = res.x
    R_opt = rotation_matrix_from_euler(alpha,beta,gamma)
    t_opt = np.array([tx,ty,tz])

    # 3) Einlesen 3D-Punkte + Quaternionen
    pts_file = f"../data/{prefix}.csv"
    pts = pd.read_csv(pts_file, sep=';')
    Qs = pts[['qx','qy','qz','qw']].to_numpy()
    starts = pts[['x','y','z']].to_numpy() / 1000

    R_quat = Rot.from_quat(Qs).as_matrix()
    # Transformiere Inspritz-Vektoren
    R_trans = np.einsum('ij,kjl->kil', R_opt, R_quat)
    dirs = R_trans[:, :, 0]  # erste Spalte als Richtungsvektor
    dirs /= np.linalg.norm(dirs, axis=1, keepdims=True)
    pos = (R_opt @ starts.T).T + t_opt

    # 4) Schnittpunkte und Ableitung von (u,v)
    us, vs, inters = [], [], []
    for p, d in zip(pos, dirs):
        X = compute_intersection_1(p, d)
        inters.append(X)
        if X is None:
            continue
        # Winkel und Höhe
        theta = np.arctan2(X[2], X[0])
        theta_unwrapped = (theta * np.linalg.norm([X[0], X[2]]) / cylinder_radius) % (2*np.pi)
        u = np.degrees(theta_unwrapped)
        v = X[1] - cylinder_end[1]
        us.append(u); vs.append(v)

    us = np.array(us) - (np.array(us).min() - 3)   # offset
    vs = np.array(vs)

    # 5) Scatter-Plot und Heatmap speichern
    out_scatter = f"../results/Diagramme/Trockenversuche/{prefix}_scatter.pdf"
    out_heatmap = f"../results/Diagramme/Trockenversuche/{prefix}_heatmap.pdf"

    # a) Scatter
    plt.figure()
    plt.scatter(vs, us, c='r', s=1)
    plt.xlabel('Zylinderhöhe in m')
    plt.ylabel('Umfangswinkel in °')
    plt.savefig(out_scatter)
    plt.close()

    # b) Dichte-Heatmap (kde)
    plt.figure()
    plt.ylim(min(us), max(us))  # Set x-axis range#
    plt.xlim(min(vs), max(vs))  # Set y-axis range
    sns.kdeplot(x=vs, y=us, fill=True, levels=100, thresh=0, cmap='inferno')
    plt.xlabel('Zylinderhöhe in m')
    plt.ylabel('Umfangswinkel in °')
    plt.savefig(out_heatmap)
    plt.close()

    print(f"[{prefix}] Scatter: {out_scatter} · Heatmap: {out_heatmap}")

Berechnung der Schnittpunkte aller Datenreihen und Erstellen aller Plots

In [4]:
if __name__ == "__main__":
    # Alle Kamera-Dateien im data-Ordner
    files_cam = glob.glob("../data/Trockenversuche_*.csv")
    prefixes = [os.path.basename(f).replace("Trockenversuche_","").replace(".csv","")
                for f in files_cam]

    for pr in prefixes:
        process_dataset(f"Trockenversuche_{pr}")


[Trockenversuche_18_03] Scatter: ../results/Diagramme/Trockenversuche/Trockenversuche_18_03_scatter.pdf · Heatmap: ../results/Diagramme/Trockenversuche/Trockenversuche_18_03_heatmap.pdf
[Trockenversuche_Kanister] Scatter: ../results/Diagramme/Trockenversuche/Trockenversuche_Kanister_scatter.pdf · Heatmap: ../results/Diagramme/Trockenversuche/Trockenversuche_Kanister_heatmap.pdf
[Trockenversuche_v_konstant] Scatter: ../results/Diagramme/Trockenversuche/Trockenversuche_v_konstant_scatter.pdf · Heatmap: ../results/Diagramme/Trockenversuche/Trockenversuche_v_konstant_heatmap.pdf
[Trockenversuche_v_konstant_mit_Nutation] Scatter: ../results/Diagramme/Trockenversuche/Trockenversuche_v_konstant_mit_Nutation_scatter.pdf · Heatmap: ../results/Diagramme/Trockenversuche/Trockenversuche_v_konstant_mit_Nutation_heatmap.pdf
[Trockenversuche_v_konstant_ohne_Nutation] Scatter: ../results/Diagramme/Trockenversuche/Trockenversuche_v_konstant_ohne_Nutation_scatter.pdf · Heatmap: ../results/Diagramme/Troc